In [112]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy
import data_generation as dg
import time

In [113]:
graph, random_solution, total_edge_length = dg.read_instance('data\data10.txt')
num_nodes = len(graph)

In [114]:
sols = []
values = []
iters = []
times = []

<span style="font-size: larger;">Classic local search</span>

In [115]:
def make_change_swap(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = random.sample(range(len(solution)), 2)
    
    # Swap the positions of the selected nodes
    new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]

    return new_solution

In [116]:
def make_change_inverse(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = sorted(random.sample(range(len(solution)), 2))
    
    # Reverse the subset of nodes between index1 and index2
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])

    return new_solution

In [117]:
def make_change_next_permutation(graph, solution):
    #based on the classic next permutation algorithm
    new_solution = deepcopy(solution)
    
    n = len(new_solution)
    
    # Find the largest index k such that a[k] < a[k+1]
    k = n - 2
    while k >= 0 and new_solution[k] >= new_solution[k + 1]:
        k -= 1

    # If no such index exists, the permutation is the last one
    if k == -1:
        return sorted(new_solution)

    # Find the largest index l greater than k such that a[k] < a[l]
    l = n - 1
    while new_solution[k] >= new_solution[l]:
        l -= 1

    # Swap a[k] and a[l]
    new_solution[k], new_solution[l] = new_solution[l], new_solution[k]

    # Reverse the sequence from a[k+1] up to and including the final element a[n-1]
    new_solution[k + 1:] = reversed(new_solution[k + 1:])

    return new_solution


In [118]:
def make_change_scramble(graph, solution):
    new_solution = deepcopy(solution)
    
    start, end = sorted(random.sample(range(len(solution)), 2))

    subset = solution[start:end+1]

    random.shuffle(subset)

    new_solution[start:end+1] = subset

    return new_solution

In [119]:
def local_search(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [120]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 3, 8, 5, 9, 7, 4, 2, 1, 0] 77 25


In [121]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 3, 8, 6, 9, 7, 4, 2, 1, 0] 77 103


In [122]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[3, 6, 8, 5, 9, 7, 4, 2, 1, 0] 77 264


In [123]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__
    results.append({'Dim': dim, 'Method': method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] <= best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,10,make_change_swap,77,0.02
1,10,make_change_inverse,77,0.02
2,10,make_change_scramble,77,0.02


In [124]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  solution_generation[best_i].__name__, 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_scramble 77 0.02 264


,Dim,Method,Value,Time,Best_Iter
0,10,make_change_scramble,77,0.02,264


In [167]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'local_search'] = values[best_i]
    df.loc[row_to_update.index, 'local_search_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'local_search': value, 'local_search_time': duration}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [126]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [77.0, 0.02, 130.67]


,Dim,Value,Time,Best_Iter
0,10,77.0,0.02,130.67


In [168]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'local_search'] = average[0]
    df.loc[row_to_update.index, 'local_search_time'] = average[1]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'local_search': average[0], 'local_search_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)

<span style="font-size: larger;">Local search with next permutation help</span>

In [128]:
sols = []
values = []
iters = []
times = []

def local_search_with_permutation(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        else:
            perm_counter = 0
            perm_limit = num_iters / (i+1)
            while new_value >= value and perm_counter < perm_limit:    
                new_solution = make_change_next_permutation(graph, new_solution)
                new_value = dg.calculate_total_edge_length(graph, new_solution)
                perm_counter += 1

            if new_value < value:
                value = new_value
                solution = deepcopy(new_solution)

                if new_value < best_value:
                    best_i = i
                    best_value = new_value
                    best_solution = deepcopy(new_solution)
              

    return best_solution, best_value, best_i

In [129]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[0, 9, 5, 2, 7, 6, 1, 8, 3, 4] 81 150


In [130]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[0, 1, 2, 4, 7, 9, 5, 8, 3, 6] 77 143


In [131]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[3, 5, 8, 6, 9, 7, 4, 2, 1, 0] 77 95


In [132]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__ + '_with_permutation'
    results.append({'Dim': dim, 'Method': method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] <= best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,10,make_change_swap_with_permutation,81,0.12
1,10,make_change_inverse_with_permutation,77,0.12
2,10,make_change_scramble_with_permutation,77,0.15


In [133]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  solution_generation[best_i].__name__ + '_with_permutation', 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_inverse 77 0.12 143


,Dim,Method,Value,Time,Best_Iter
0,10,make_change_inverse_with_permutation,77,0.12,143


In [134]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'local_search_perm'] = values[best_i]
    df.loc[row_to_update.index, 'local_search_perm_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'local_search_perm': value, 'local_search_perm_time': duration}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [135]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [78.33, 0.13, 129.33]


,Dim,Value,Time,Best_Iter
0,10,78.33,0.13,129.33


In [136]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'local_search_perm'] = average[0]
    df.loc[row_to_update.index, 'local_search_perm_time'] = average[1]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'local_search_perm': average[0], 'local_search_perm_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)

<span style="font-size: larger;">Simulated annealing</span>

In [137]:
sols = []
values = []
iters = []
times = []

def simulated_annealing(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(1, num_iters + 1):
        #print(solution, value)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        #elif random.random() < 1 / (i**0.5):
        elif random.random() < 1 / i:
            #print('divs')
            value = new_value
            solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [138]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[0, 9, 6, 2, 7, 1, 4, 8, 5, 3] 81 77


In [139]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[0, 1, 4, 2, 7, 9, 5, 8, 6, 3] 77 82


In [140]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 3, 8, 6, 9, 7, 2, 4, 1, 0] 77 244


In [141]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__
    results.append({'Dim': dim, 'Method': 'simulated_annealing' + ', '  + method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] <= best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,10,"simulated_annealing, make_change_swap",81,0.02
1,10,"simulated_annealing, make_change_inverse",77,0.02
2,10,"simulated_annealing, make_change_scramble",77,0.03


In [142]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  'simulated_annealing' + ', '  + solution_generation[best_i].__name__, 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_inverse 77 0.02 82


,Dim,Method,Value,Time,Best_Iter
0,10,"simulated_annealing, make_change_inverse",77,0.02,82


In [143]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'sim_annealing'] = values[best_i]
    df.loc[row_to_update.index, 'sim_annealing_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'sim_annealing': value, 'sim_annealing_time': duration}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [144]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [78.33, 0.02, 134.33]


,Dim,Value,Time,Best_Iter
0,10,78.33,0.02,134.33


In [145]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'sim_annealing'] = average[0]
    df.loc[row_to_update.index, 'sim_annealing_time'] = average[1]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'sim_annealing_perm': average[0], 'sim_annealing_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)

<span style="font-size: larger;">Variable neigborhood search</span>

In [146]:
def shaking_swap(graph, solution, k):
    new_solution = deepcopy(solution)
    selected = random.sample(range(len(solution)), 2*k)
    for i in range(0,len(selected), 2):
        index1, index2 = selected[i], selected[i+1]
        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]
    return new_solution    
    

In [147]:
def shaking_inverse(graph, solution, k):
    new_solution = deepcopy(solution)
    index1 = random.randint(0, len(solution))
    if(index1 + k > len(solution)):
        index1 = len(solution) - k
    index2 = index1 + k                       
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])  

    return new_solution

In [148]:
def vns(graph, random_solution, value, num_iters, change_func, shaking_func, local_search_func, k_min, k_max, move_prob):
    solution = deepcopy(random_solution)
    best_i = None
    for i in range(num_iters):
        for k in range(k_min, k_max):
            #print('vns: ', solution, value, i)
            new_solution = shaking_func(graph, solution, k) #diversification  

            new_value = dg.calculate_total_edge_length(graph, new_solution)

            #print('post shaking: ', new_solution, new_value, i)
            new_solution, new_value, _= local_search_func(graph, new_solution, total_edge_length, 10, change_func)

            if new_value < value or (new_value == value and random.random() < move_prob):
                if(new_value < value):
                    best_i = i

                value = new_value

                solution = deepcopy(new_solution)
                
    return solution, value, best_i

In [149]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 3, 8, 5, 9, 7, 2, 4, 1, 0] 77 61


In [150]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 3, 8, 5, 9, 7, 4, 2, 1, 0] 77 41


In [151]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[3, 6, 8, 5, 9, 7, 2, 4, 1, 0] 77 56


In [152]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 3, 8, 5, 9, 7, 2, 4, 1, 0] 77 81


In [153]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[3, 6, 8, 5, 9, 7, 2, 4, 1, 0] 77 36


In [154]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[3, 6, 8, 5, 9, 7, 2, 4, 1, 0] 77 15


In [155]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[3, 5, 8, 6, 9, 7, 4, 2, 1, 0] 77 20


In [156]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[3, 5, 8, 6, 9, 7, 2, 4, 1, 0] 77 38


In [157]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[0, 1, 4, 2, 7, 9, 6, 8, 3, 5] 77 52


In [158]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[0, 1, 4, 2, 7, 9, 6, 8, 5, 3] 77 32


In [159]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[5, 3, 8, 6, 9, 7, 2, 4, 1, 0] 77 11


In [160]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[3, 5, 8, 6, 9, 7, 2, 4, 1, 0] 77 126


In [161]:
n = len(graph)

oc = False

shakings = [shaking_swap, shaking_inverse]
changes_funcs = [make_change_swap, make_change_inverse, make_change_scramble]
search_funcs = [local_search, local_search_with_permutation]

number_of_combinations = 0

sols = []
values =[]
times = []

methods = []

results = []

best = float('inf')
best_i = -1

average = [0,0]

k_min = int(0.1 * num_nodes)
k_max = int(0.3 * num_nodes)
for shake in shakings:
    for change in changes_funcs:
        for search in search_funcs:
            if shake.__name__ == 'shaking_inverse':
                k_min = int(0.3 * num_nodes)
                k_max = int(0.6 * num_nodes)

            start = time.time()    
            sol, value, iter = vns(graph, 
                                   random_solution, 
                                   total_edge_length, 
                                   500, 
                                   change_func=change, 
                                   shaking_func=shake, 
                                   local_search_func = search, 
                                   k_min=k_min,
                                   k_max=k_max, 
                                   move_prob=0.4)
            end = time.time()
            duration = float("{:.2f}".format(end - start))
            
            print(shake.__name__, change.__name__, search.__name__, ' : ', sol, value, iter, '\n')
            
            k_min = int(0.1 * num_nodes)
            k_max = int(0.3 * num_nodes)

            sols.append(sol)
            values.append(value)
            iters.append(iter)
            times.append(duration)

            methods.append(str(shake.__name__ + ', ' + change.__name__ + ', ' + search.__name__))

            results.append({'Dim': num_nodes, 'Method': methods[number_of_combinations], 'Value': abs(value), 'Time': duration})

            if value <= best:
                if duration <= times[best_i]:    
                    best = value
                    best_i = number_of_combinations

            average[0] += value
            average[1] += duration

            number_of_combinations += 1    


shaking_swap make_change_swap local_search  :  [3, 6, 8, 5, 9, 7, 2, 4, 1, 0] 77 15 

shaking_swap make_change_swap local_search_with_permutation  :  [3, 5, 8, 6, 9, 7, 4, 2, 1, 0] 77 6 

shaking_swap make_change_inverse local_search  :  [3, 5, 8, 6, 9, 7, 4, 2, 1, 0] 77 101 

shaking_swap make_change_inverse local_search_with_permutation  :  [3, 5, 8, 6, 9, 7, 2, 4, 1, 0] 77 41 

shaking_swap make_change_scramble local_search  :  [0, 1, 2, 4, 7, 9, 6, 8, 5, 3] 77 22 

shaking_swap make_change_scramble local_search_with_permutation  :  [0, 1, 2, 4, 7, 9, 5, 8, 6, 3] 77 65 

shaking_inverse make_change_swap local_search  :  [0, 1, 2, 4, 7, 9, 5, 8, 3, 6] 77 39 

shaking_inverse make_change_swap local_search_with_permutation  :  [0, 1, 4, 2, 7, 9, 6, 8, 5, 3] 77 24 

shaking_inverse make_change_inverse local_search  :  [6, 3, 8, 5, 9, 7, 4, 2, 1, 0] 77 17 

shaking_inverse make_change_inverse local_search_with_permutation  :  [5, 3, 8, 6, 9, 7, 2, 4, 1, 0] 77 15 

shaking_inverse make_ch

In [162]:
df = pd.DataFrame(results)
display(df)
df.to_csv('comparison_tables/vns.csv', mode='a', header=not pd.io.common.file_exists('comparison_tables/vns.csv'), index=False)

,Dim,Method,Value,Time
0,10,"shaking_swap, make_change_swap, local_search",77,0.42
1,10,"shaking_swap, make_change_swap, local_search_w...",77,1.22
2,10,"shaking_swap, make_change_inverse, local_search",77,0.43
3,10,"shaking_swap, make_change_inverse, local_searc...",77,1.26
4,10,"shaking_swap, make_change_scramble, local_search",77,0.58
5,10,"shaking_swap, make_change_scramble, local_sear...",77,1.37
6,10,"shaking_inverse, make_change_swap, local_search",77,0.61
7,10,"shaking_inverse, make_change_swap, local_searc...",77,1.75
8,10,"shaking_inverse, make_change_inverse, local_se...",77,0.63
9,10,"shaking_inverse, make_change_inverse, local_se...",77,1.84


In [163]:
print('best:', methods[best_i], values[best_i], times[best_i])
df_best = pd.DataFrame({'Dim': num_nodes, 'Method':  methods[best_i], 'Value': values[best_i], 'Time': times[best_i]}, index=[0])
display(df_best)

best: shaking_swap, make_change_swap, local_search 77 0.42


,Dim,Method,Value,Time
0,10,"shaking_swap, make_change_swap, local_search",77,0.42


In [164]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'vns'] = values[best_i]
    df.loc[row_to_update.index, 'vns_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'vns': value, 'vns_time': duration}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [165]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': num_nodes, 'Value': average[0], 'Time': average[1]}, index=[0])
display(df_avg)

average: [77.0, 1.08]


,Dim,Value,Time
0,10,77.0,1.08


In [166]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'vns'] = average[0]
    df.loc[row_to_update.index, 'vns_time'] = average[1]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'vns_perm': average[0], 'vns_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)